In [1]:
!pip install transformers datasets faiss-gpu faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import faiss
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from datasets import load_dataset
import tensorflow as tf
import json
from tqdm import tqdm

# Dataset and model configuration
DATASET_NAME = "QuyenAnhDE/Diseases_Symptoms"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
INDEX_FILE = "diseases_symptoms_index.faiss"
EMBEDDINGS_FILE = "diseases_symptoms_embeddings.npy"
DOCS_FILE = "diseases_symptoms_docs.json"

# Set up device
device = tf.device('/GPU:0') if tf.config.list_physical_devices('GPU') else tf.device('/CPU:0')
print(f"Using device: {device}")

# Load the Diseases_Symptoms dataset
print("Loading dataset...")
dataset = load_dataset(DATASET_NAME, split="train")



Using device: <tensorflow.python.eager.context._EagerDeviceContext object at 0x7c7f39d625c0>
Loading dataset...


Repo card metadata block was not found. Setting CardData to empty.


In [4]:
dataset

Dataset({
    features: ['Code', 'Name', 'Symptoms', 'Treatments'],
    num_rows: 400
})

In [6]:

# Preprocess dataset into a format suitable for retrieval
def preprocess_dataset(dataset):
    """Extract and preprocess diseases and symptoms data."""
    documents = []
    for entry in dataset:
        disease = entry["Name"]
        symptoms = entry["Symptoms"]
        documents.append({
            "disease": disease,
            "symptoms": symptoms,
            "combined": f"Disease: {disease}. Symptoms: {symptoms}"
        })
    return documents

print("Processing dataset...")
documents = preprocess_dataset(dataset)

# Save the documents for later use
with open(DOCS_FILE, "w", encoding="utf-8") as f:
    json.dump(documents, f)
print(f"Processed and saved {len(documents)} records.")


Processing dataset...
Processed and saved 400 records.


In [8]:

# Load embedding model
print("Loading embedding model...")
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
model = TFAutoModel.from_pretrained(EMBEDDING_MODEL_NAME)

# Generate embeddings for each document
def generate_embeddings(documents, batch_size=16):
    """Generate embeddings for documents."""
    embeddings = []
    for i in tqdm(range(0, len(documents), batch_size), desc="Generating embeddings"):
        batch_texts = [doc["combined"] for doc in documents[i:i + batch_size]]
        inputs = tokenizer(batch_texts, return_tensors="tf", padding=True, truncation=True, max_length=512)
        with tf.device('/GPU:0'):
            outputs = model(inputs)
        batch_embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy()
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

print("Generating embeddings...")
embeddings = generate_embeddings(documents)

# Save embeddings for reuse
np.save(EMBEDDINGS_FILE, embeddings)
print(f"Saved embeddings for {len(embeddings)} records.")

# Build and save FAISS index
print("Building FAISS index...")
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))
faiss.write_index(index, INDEX_FILE)
print(f"Saved FAISS index to {INDEX_FILE}.")


Loading embedding model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Generating embeddings...


Generating embeddings: 100%|██████████| 25/25 [00:04<00:00,  6.11it/s]

Saved embeddings for 400 records.
Building FAISS index...
Saved FAISS index to diseases_symptoms_index.faiss.


In [9]:

# Load retrieval components
def load_retrieval_components():
    """Load documents, embeddings, and FAISS index."""
    with open(DOCS_FILE, "r", encoding="utf-8") as f:
        docs = json.load(f)
    embeddings = np.load(EMBEDDINGS_FILE)
    index = faiss.read_index(INDEX_FILE)
    return docs, embeddings, index

# Retrieve top-k relevant documents
def retrieve_documents(query, docs, index, top_k=5):
    """Retrieve relevant documents based on a query."""
    inputs = tokenizer([query], return_tensors="tf", padding=True, truncation=True, max_length=512)
    with tf.device('/GPU:0'):
        outputs = model(inputs)
    query_embedding = tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy().astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    return [docs[idx] for idx in indices[0]]

# Test retrieval system
print("Testing retrieval system...")
docs, embeddings, index = load_retrieval_components()

test_query = "What are the symptoms of diabetes?"
retrieved_docs = retrieve_documents(test_query, docs, index)

print(f"Query: {test_query}")
print("\nRetrieved Documents:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"{i}. Disease: {doc['disease']}")
    print(f"   Symptoms: {doc['symptoms']}")

print("Retrieval system ready.")

Testing retrieval system...
Query: What are the symptoms of diabetes?

Retrieved Documents:
1. Disease: Insulin Overdose
   Symptoms: Low blood sugar (hypoglycemia) symptoms (e.g., confusion, sweating)
2. Disease: Diabetic Kidney Disease
   Symptoms: High blood pressure, increased need to urinate, swelling in the legs and ankles, fatigue, nausea, loss of appetite
3. Disease: Type 2 Diabetes
   Symptoms: Fatigue, Increased hunger, Slow healing of wounds
4. Disease: Hyperosmotic Hyperketotic State
   Symptoms: Increased blood sugar levels, increased ketone production, excessive thirst, frequent urination, fatigue, confusion, fruity breath odor
5. Disease: Chronic Kidney Disease
   Symptoms: Fatigue, swelling of the legs or ankles, decreased appetite, difficulty concentrating, increased urination or urine changes, blood in urine, high blood pressure
Retrieval system ready.
